In [ ]:
!pip install numpy matplotlib plotly pandas

In [ ]:
import sys

!{sys.executable} -m pip install librosa

In [ ]:
!pip install nbformat>=4.2.0

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import json
import librosa
from scipy.fft import fft, fftfreq
from scipy.signal import spectrogram

# Get data

In [2]:
npz_data = np.load('../../data/48k_data/training.npz')
meta_data = json.load(open("../../data/48k_data/meta.json", 'r'))

In [ ]:
# Print the keys to understand the file structure
print(npz_data.keys())

- X: audio data
- y: label
- groups:

labels:  
- "Disappointment": 0
- "Boring": 1
- "Chanting/Singing": 2
- "Celebration": 3
- "Excitement": 4

# Sample data

In [ ]:
idx = 0
print(npz_data["X"][idx])
print(npz_data["y"][idx])
print(npz_data["groups"][idx])

In [ ]:
id2label = {v:k for k,v in meta_data['labels'].items()}
id2label

In [6]:
# get one example of every label
examples = {}

for i, y in enumerate(npz_data["y"]):
    label = id2label[y]
    if label not in examples:
        examples[label] = {"Index": i, "Signal": npz_data["X"][i], "LabelID": y, "LabelName": label}

df_examples = pd.DataFrame.from_dict(examples).T

# Plot audio waveform

In [ ]:
idx = 0

label_id = id2label[npz_data["y"][idx]]

plt.figure(figsize=(12, 4))

time = np.arange(len(npz_data["X"][idx])) / meta_data["sample_rate"]

plt.plot(time, npz_data["X"][idx])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'Audio Waveform (Index: {idx}, Label: {label_id})')  # Titel mit Index und Label
plt.grid()
plt.show()


# Spectrogram

In [ ]:
idx = 0

frequencies, times, Sxx = spectrogram(npz_data["X"][idx], meta_data["sample_rate"])
plt.figure(figsize=(12, 6))
plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud')
plt.colorbar(label='Power (dB)')
plt.ylabel("Frequency (Hz)")
plt.xlabel("Time (s)")
plt.title("Spectrogram")
plt.show()

### Melspectrogram

In [ ]:
idx = 50

S = librosa.feature.melspectrogram(
    y=npz_data['X'][idx],
    sr=meta_data["sample_rate"],
    power=1
)

librosa.display.specshow(S, x_axis='time', y_axis='mel')

In [ ]:
for _, row in df_examples.iterrows():
    S = librosa.feature.melspectrogram(
        y=row["Signal"],
        sr=meta_data["sample_rate"],
        power=1
    )

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S, sr=meta_data["sample_rate"], x_axis='time', y_axis='mel')
    plt.title(f"Mel-Spektrogramm - {row["LabelName"]} (Index: {row["Index"]})")
    plt.show()

# Frequency spectrum

In [ ]:
idx = 0

min_amplitude = 1e-6

N = len(npz_data["X"][idx])  # Number of Samples
T = 1.0 / meta_data["sample_rate"]
yf = fft(npz_data["X"][idx]) 
xf = fftfreq(N, T)[:N // 2] / 1000  # Frequency in kHz
max_frequency = meta_data["sample_rate"] / 2 / 1000  # Nyquist-Frequency in kHz

amplitudes = 2.0 / N * np.abs(yf[:N // 2])

filtered_xf = xf[amplitudes > min_amplitude]
filtered_amplitudes = amplitudes[amplitudes > min_amplitude]

plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(xf, amplitudes)
plt.grid()
plt.xlabel("Frequency (kHz)")
plt.ylabel("Amplitude")
plt.title("Frequency Spectrum")
plt.xlim(0, max_frequency)

plt.subplot(2, 1, 2)
plt.plot(filtered_xf, filtered_amplitudes)
plt.grid()
plt.xlabel("Frequency (kHz)")
plt.ylabel("Amplitude")
plt.title(f"Filtered Frequencies (Amplitude > {min_amplitude})")
plt.tight_layout()

plt.show()


In [ ]:
min_amplitude = 1e-6

for _, row in df_examples.iterrows():
    signal = row["Signal"]
    sample_rate = meta_data["sample_rate"]
    
    N = len(signal)
    T = 1.0 / sample_rate
    yf = fft(signal)
    xf = fftfreq(N, T)[:N // 2] / 1000
    amplitudes = 2.0 / N * np.abs(yf[:N // 2])
    
    filtered_xf = xf[amplitudes > min_amplitude]
    filtered_amplitudes = amplitudes[amplitudes > min_amplitude]
    max_frequency = sample_rate / 2 / 1000
    
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(xf, amplitudes)
    plt.grid()
    plt.xlabel("Frequency (kHz)")
    plt.ylabel("Amplitude")
    plt.title(f"Frequency Spectrum - {row['LabelName']} (Index: {row['Index']})")
    plt.xlim(0, max_frequency)
    
    plt.subplot(2, 1, 2)
    plt.plot(filtered_xf, filtered_amplitudes)
    plt.grid()
    plt.xlabel("Frequency (kHz)")
    plt.ylabel("Amplitude")
    plt.title(f"Filtered Frequencies (Amplitude > {min_amplitude})")
    plt.tight_layout()
    
    plt.show()